# Introduction

<span class="xp-tag-icon is-shown-complete docon docon-check" style="box-sizing: inherit; outline-color: inherit; font-family: docons; font-size: 9px; speak: none; font-variant-numeric: normal; font-variant-east-asian: normal; text-align: center; direction: ltr; -webkit-font-smoothing: antialiased; line-height: 16px; display: inline-block; z-index: 3; color: var(--theme-success-invert); animation: 0.8s ease 0s 1 normal none running scaleIn; position: absolute; top: 1px; left: -5px;"><span class="visually-hidden" style="box-sizing: inherit; outline-color: inherit; clip: rect(1px, 1px, 1px, 1px); clip-path: inset(50%); height: 1px; width: 1px; overflow-wrap: normal; border: 0px; margin: -1px; padding: 0px; position: absolute; overflow: hidden;">Completed</span></span>

In many cases, data analysts and business users simply need to retrieve data from a database for reporting or analysis. However, when developing an application, or even during some complex analysis, you may need to insert, modify, or delete data.

In this module, you'll learn how to:

- Insert data into a table
- Generate automatic values
- Update data in a table
- Delete data from a table
- Merge data based on multiple tables

# Insert data

<span class="xp-tag-icon is-shown-complete docon docon-check" style="box-sizing: inherit; outline-color: inherit; font-family: docons; font-size: 9px; speak: none; font-variant-numeric: normal; font-variant-east-asian: normal; text-align: center; direction: ltr; -webkit-font-smoothing: antialiased; line-height: 16px; display: inline-block; z-index: 3; color: var(--theme-success-invert); animation: 0.8s ease 0s 1 normal none running scaleIn; position: absolute; top: 1px; left: -5px;"><span class="visually-hidden" style="box-sizing: inherit; outline-color: inherit; clip: rect(1px, 1px, 1px, 1px); clip-path: inset(50%); height: 1px; width: 1px; overflow-wrap: normal; border: 0px; margin: -1px; padding: 0px; position: absolute; overflow: hidden;">Completed</span></span>

Transact-SQL provides multiple ways to insert rows into a table.

## The INSERT statement

The INSERT statement is used to add one or more rows to a table. There are several forms of the statement.

The basic syntax of a simple INSERT statement is shown below:

SQLCopy

```
INSERT [INTO] <Table> [(column_list)]
VALUES ([ColumnName or an expression or DEFAULT or NULL],…n)

```

With this form of the INSERT statement, called INSERT VALUES, you can specify the columns that will have values placed in them and the order in which the data will be presented for each row inserted into the table. The column\_list is optional but recommended. Without the column\_list, the INSERT statement will expect a value for every column in the table in the order in which the columns were defined. You can also provide the values for those columns as a comma-separated list.

When listing values, the keyword DEFAULT means a predefined value, that was specified when the table was created, will be used. There are three ways a default can be determined:

- If a column has been defined to have an automatically generated value, that value will be used. Autogenerated values will be discussed later in this module.
- When a table is created, a default value can be supplied for a column, and that value will be used if DEFAULT is specified.
- If a column has been defined to allow NULL values, and the column isn't an autogenerated column and doesn't have a default defined, NULL will be inserted as a DEFAULT.

The details of table creation are beyond the scope of this module. However, it is often useful to see what columns are in a table. The easiest way is to just execute a SELECT statement on the table without returning any rows. By using a WHERE condition that can never be TRUE, no rows can be returned.

SQLCopy

```
SELECT * FROM Sales.Promotion
WHERE 1 = 0;

```

This statement will show you all the columns and their names, but won't show the data types or any properties, such as whether NULLs are allowed, or if there is a default values specified. An example of the output from the query might look like this:

PromotionName

StartDate

ProductModelID

Discount

Notes

To insert data into this table, you can use the INSERT statement as shown here.

SQLCopy

```
INSERT INTO Sales.Promotion (PromotionName,StartDate,ProductModelID,Discount,Notes)
VALUES
('Clearance Sale', '01/01/2021', 23, 0.1, '10% discount');

```

For this example above, the column list can be omitted, because we're supplying a value for every column in the correct order:

SQLCopy

```
INSERT INTO Sales.Promotion
VALUES
('Clearance Sale', '01/01/2021', 23, 0.1, '10% discount');

```

Suppose that the table is defined such that a default value of the current date is applied to the StartDate column, and the Notes column allows NULL values. You can indicate that you want to use these values explicitly, like this:

SQLCopy

```
INSERT INTO Sales.Promotion
VALUES
('Pull your socks up', DEFAULT, 24, 0.25, NULL);

```

Alternatively, you can omit values in the INSERT statement, in which case the default value will be used if defined, and if there is no default value but the column allows NULLs, then a NULL will be inserted. If you’re not supplying values for all columns, you must have a column list indicated which column values you're supplying.

SQLCopy

```
INSERT INTO Sales.Promotion (PromotionName, ProductModelID, Discount)
VALUES
('Caps Locked', 2, 0.2);

```

In addition to inserting a single row at a time, the INSERT VALUES statement can be used to insert multiple rows by providing multiple comma-separated sets of values. The sets of values are also separated by commas, like this:

SQLCopy

```
(col1_val,col2_val,col3_val),
(col1_val,col2_val,col3_val)

```

This list of values is known as a table value constructor. Here’s an example of inserting two more rows into our table with a table value constructor:

SQLCopy

```
INSERT INTO Sales.Promotion
VALUES
('The gloves are off!', DEFAULT, 3, 0.25, NULL),
('The gloves are off!', DEFAULT, 4, 0.25, NULL);

```

## INSERT ... SELECT

In addition to specifying a literal set of values in an INSERT statement, T-SQL also supports using the results of other operations to provide values for INSERT. You can use the results of a SELECT statement or the output of a stored procedure to supply the values for the INSERT statement.

To use the INSERT with a nested SELECT, build a SELECT statement to replace the VALUES clause. With this form, called INSERT SELECT, you can insert the set of rows returned by a SELECT query into a destination table. The use of INSERT SELECT presents the same considerations as INSERT VALUES:

- You may optionally specify a column list following the table name.
- You must provide column values or DEFAULT, or NULL, for each column.

The following syntax illustrates the use of INSERT SELECT:

SQLCopy

```
INSERT [INTO] <table or view> [(column_list)]
SELECT <column_list> FROM <table_list>...;

```

 Note

Result sets from stored procedures (or even dynamic batches) may also be used as input to an INSERT statement. This form of INSERT, called INSERT EXEC, is conceptually similar to INSERT SELECT and will present the same considerations. However, stored procedures can return multiple result sets, so extra care is needed.

The following example inserts multiple rows for a new promotion named _Get Framed_ by retrieving the model ID and model name from the Production.ProductModel, table for every model that contains "frame" in its name.

SQLCopy

```
INSERT INTO Sales.Promotion (PromotionName, ProductModelID, Discount, Notes)
SELECT DISTINCT 'Get Framed', m.ProductModelID, 0.1, '10% off ' + m.Name
FROM Production.ProductModel AS m
WHERE m.Name LIKE '%frame%';

```

Unlike a subquery, the nested SELECT used with an INSERT isn't enclosed in parentheses.

## SELECT ... INTO

Another option for inserting rows, which is similar to INSERT SELECT, is the SELECT INTO statement. The biggest difference between INSERT SELECT and SELECT INTO is that SELECT INTO cannot be used to insert rows into an existing table, because it always creates a new table that is based on the result of the SELECT. Each column in the new table will have the same name, data type, and nullability as the corresponding column (or expression) in the SELECT list.

To use SELECT INTO, add INTO \<new\_table\_name\> in the SELECT clause of the query, just before the FROM clause. Here’s an example that extracts data from the Sales.SalesOrderHeader table into a new table named Sales.Invoice..

SQLCopy

```
SELECT SalesOrderID, CustomerID, OrderDate, PurchaseOrderNumber, TotalDue
INTO Sales.Invoice
FROM Sales.SalesOrderHeader;

```

A SELECT INTO will fail if there already is a table with the name specified after INTO. After the table is created, it can be treated like any other table. You can select from it, join it to other tables, or insert more rows into it.

# Generate automatic values

<span class="xp-tag-icon is-shown-complete docon docon-check" style="box-sizing: inherit; outline-color: inherit; font-family: docons; font-size: 9px; speak: none; font-variant-numeric: normal; font-variant-east-asian: normal; text-align: center; direction: ltr; -webkit-font-smoothing: antialiased; line-height: 16px; display: inline-block; z-index: 3; color: var(--theme-success-invert); animation: 0.8s ease 0s 1 normal none running scaleIn; position: absolute; top: 1px; left: -5px;"><span class="visually-hidden" style="box-sizing: inherit; outline-color: inherit; clip: rect(1px, 1px, 1px, 1px); clip-path: inset(50%); height: 1px; width: 1px; overflow-wrap: normal; border: 0px; margin: -1px; padding: 0px; position: absolute; overflow: hidden;">Completed</span></span>

You may need to automatically generate sequential values for one column in a specific table. Transact-SQL provides two ways to do this: use the IDENTITY property with a specific column in a table, or define a SEQUENCE object and use values generated by that object.

## The IDENTITY property

To use the IDENTITY property, define a column using a numeric data type with a scale of 0 (meaning whole numbers only) and include the IDENTITY keyword. The allowable types include all integer types and decimal types where you explicitly give a scale of 0.

An optional seed (starting value), and an increment (step value) can also be specified. Leaving out the seed and increment will set them both to 1.

 Note

The IDENTITY property is specified in place of specifying NULL or NOT NULL in the column definition. Any column with the IDENTITY property is automatically not nullable. You can specify NOT NULL just for self-documentation, but if you specify the column as NULL (meaning nullable), the table creation statement will generate an error.

Only one column in a table may have the IDENTITY property set; it's frequently used as either the PRIMARY KEY or an alternate key.

The following code shows the creation of the Sales.Promotion table used in the previous section examples, but this time with an identity column named PromotionID as the primary key:

SQLCopy

```
CREATE TABLE Sales.Promotion
(
PromotionID int IDENTITY PRIMARY KEY,
PromotionName varchar(20),
StartDate datetime NOT NULL DEFAULT GETDATE(),
ProductModelID int NOT NULL REFERENCES Production.ProductModel(ProductModelID),
Discount decimal(4,2) NOT NULL,
Notes nvarchar(max) NULL
);

```

 Note

The full details of the CREATE TABLE statement are beyond the scope of this module.

### Inserting data into an identity column

When the IDENTITY property is defined for a column, INSERT statements into the table generally don't specify a value for the IDENTITY column. The database engine generates a value using the next available value for the column.

For example, you could insert a row into the Sales.Promotion table without specifying a value for the PromotionID column:

SQLCopy

```
INSERT INTO Sales.Promotion
VALUES
('Clearance Sale', '01/01/2021', 23, 0.10, '10% discount')

```

Notice that even though the VALUES clause doesn't include a value for the PromotionID column, you don't need to specify a column list in the INSERT clause - Identity columns are exempt from this requirement.

If this row is the first one inserted into the table, the result is a new row like this:

PromotionID

PromotionName

StartDate

ProductModelID

Discount

Notes

1

Clearance Sale

2021-01-01T00:00:00

23

0.1

10% discount

When the table was created, no seed or increment values were set for the IDENTITY column, so the first row is inserted with a value of 1. The next row to be inserted will be assigned a PromotionID value of 2, and so on.

### Retrieving an identity value

To return the most recently assigned IDENTITY value within the same session and scope, use the SCOPE\_IDENTITY function; like this:

SQLCopy

```
SELECT SCOPE_IDENTITY();

```

The SCOPE\_IDENTITY function returns the most recent identity value generated in the current scope for any table. If you need the latest identity value in a specific table, you can use the IDENT\_CURRENT function, like this:

SQLCopy

```
SELECT IDENT_CURRENT('Sales.Promotion');

```

### Overriding identity values

If you want to override the automatically generated value and assign a specific value to the IDENTITY column, you first need to enable identity inserts by using the SET IDENTITY INSERT _table\_name_ ON statement. With this option enabled, you can insert an explicit value for the identity column, just like any other column. When you're finished, you can use the SET IDENTITY INSERT _table\_name_ OFF statement to resume using automatic identity values, using the last value you explicitly entered as a seed.

SQLCopy

```
SET IDENTITY_INSERT SalesLT.Promotion ON;

INSERT INTO SalesLT.Promotion (PromotionID, PromotionName, ProductModelID, Discount)
VALUES
(20, 'Another short sale',37, 0.3);

SET IDENTITY_INSERT SalesLT.Promotion OFF;

```

As you've learned, the IDENTITY property is used to generate a sequence of values for a column within a table. However, the IDENTITY property isn't suitable for coordinating values across multiple tables within a database. For example, suppose your organization differentiates between direct sales and sales to resellers, and wants to store data for these sales in separate tables. Both kinds of sale may need a unique invoice number, and you may want to avoid duplicating the same value for two different kinds of sale. One solution for this requirement is to maintain a pool of unique sequential values across both tables.

### Reseeding an identity column

Occasionally, you'll need to reset or skip identity values for the column. To do this, you'll be "reseeding" the column using the DBCC CHECKIDENT function. You can use this to skip many values, or to reset the next identity value to 1 after you've deleted all of the rows in the table. For full details using DBCC CHECKIDENT, see the [Transact-SQL reference documentation](https://learn.microsoft.com/en-us/sql/t-sql/database-console-commands/dbcc-checkident-transact-sql).

## SEQUENCE

In Transact-SQL, you can use a sequence object to define new sequential values independently of a specific table. A sequence object is created using the CREATE SEQUENCE statement, optionally supplying the data type (must be an integer type or decimal or numeric with a scale of 0), the starting value, an increment value, a maximum value, and other options related to performance.

SQLCopy

```
CREATE SEQUENCE Sales.InvoiceNumber AS INT
START WITH 1000 INCREMENT BY 1;

```

To retrieve the next available value from a sequence, use the NEXT VALUE FOR construct, like this:

SQLCopy

```
INSERT INTO Sales.ResellerInvoice
VALUES
(NEXT VALUE FOR Sales.InvoiceNumber, 2, GETDATE(), 'PO12345', 107.99);

```

### IDENTITY or SEQUENCE

When deciding whether to use IDENTITY columns or a SEQUENCE object for auto-populating values, keep the following points in mind:

- Use SEQUENCE if your application requires sharing a single series of numbers between multiple tables or multiple columns within a table.
    
- SEQUENCE allows you to sort the values by another column. The NEXT VALUE FOR construct can use the OVER clause to specify the sort column. The OVER clause guarantees that the values returned are generated in the order of the OVER clause's ORDER BY clause. This functionality also allows you to generate row numbers for rows as they’re being returned in a SELECT. In the following example, the Production.Product table is sorted by the Name column, and the first returned column is a sequential number.
    
    SQLCopy
    
    ```
    SELECT NEXT VALUE FOR dbo.Sequence OVER (ORDER BY Name) AS NextID,
        ProductID,
        Name
    FROM Production.Product;
    
    ```
    
    Even though the previous statement was just selecting SEQUENCE values to display, the values are still being 'used up' and the displayed SEQUENCE values will no longer be available. If you run the above SELECT multiple times, you'll get different SEQUENCE values each time.
    
- Use SEQUENCE if your application requires multiple numbers to be assigned at the same time. For example, an application needs to reserve five sequential numbers. Requesting identity values could result in gaps in the series if other processes were simultaneously issued numbers. You can use the sp\_sequence\_get\_range system procedure to retrieve several numbers in the sequence at once.
    
- SEQUENCE allows you to change the specification of the sequence, such as the increment value.
    
- IDENTITY values are protected from updates. If you try to update a column with the IDENTITY property, you'll get an error.

# Update data

<span class="xp-tag-icon is-shown-complete docon docon-check" style="box-sizing: inherit; outline-color: inherit; font-family: docons; font-size: 9px; speak: none; font-variant-numeric: normal; font-variant-east-asian: normal; text-align: center; direction: ltr; -webkit-font-smoothing: antialiased; line-height: 16px; display: inline-block; z-index: 3; color: var(--theme-success-invert); animation: 0.8s ease 0s 1 normal none running scaleIn; position: absolute; top: 1px; left: -5px;"><span class="visually-hidden" style="box-sizing: inherit; outline-color: inherit; clip: rect(1px, 1px, 1px, 1px); clip-path: inset(50%); height: 1px; width: 1px; overflow-wrap: normal; border: 0px; margin: -1px; padding: 0px; position: absolute; overflow: hidden;">Completed</span></span>

The UPDATE statement in T-SQL is used to change existing data in a table. UPDATE operates on a set of rows, either defined by a condition in a WHERE clause or defined in a join. The UPDATE statement has a SET clause that specifies which columns are to be modified. The SET clause one or more columns, separated by commas, and supplies new values to those columns. The WHERE clause in an UPDATE statement has the same structure as a WHERE clause in a SELECT statement.

 Note

It’s important to note that an UPDATE without a corresponding WHERE clause or a join, will update all the rows in a table. Use the UPDATE statement with caution.

The basic syntax of an UPDATE statement is shown below.

SQLCopy

```
UPDATE <TableName>
SET 
<ColumnName> = { expression | DEFAULT | NULL }
{,…n}
WHERE <search_conditions>;

```

The following example shows the UPDATE statement used to modify the notes for a promotion:

SQLCopy

```
UPDATE Sales.Promotion
SET Notes = '25% off socks'
WHERE PromotionID = 2;

```

You can modify multiple columns in the SET clause. For example, the following UPDATE statement modified both the Discount and Notes fields for all rows where the promotion name is "Get Framed":

SQLCopy

```
UPDATE Sales.Promotion
SET Discount = 0.2, Notes = REPLACE(Notes, '10%', '20%')
WHERE PromotionName = 'Get Framed';

```

The UPDATE statement also supports a FROM clause, enabling you to modify data based on the results of a query. For example, the following code updates the Sales.Promotion table using values retrieved from the Product.ProductModel table.

SQLCopy

```
UPDATE Sales.Promotion
SET Notes = FORMAT(Discount, 'P') + ' off ' + m.Name
FROM Product.ProductModel AS m
WHERE Notes IS NULL
    AND Sales.Promotion.ProductModelID = m.ProductModelID;

```

# Delete data

<span class="xp-tag-icon is-shown-complete docon docon-check" style="box-sizing: inherit; outline-color: inherit; font-family: docons; font-size: 9px; speak: none; font-variant-numeric: normal; font-variant-east-asian: normal; text-align: center; direction: ltr; -webkit-font-smoothing: antialiased; line-height: 16px; display: inline-block; z-index: 3; color: var(--theme-success-invert); animation: 0.8s ease 0s 1 normal none running scaleIn; position: absolute; top: 1px; left: -5px;"><span class="visually-hidden" style="box-sizing: inherit; outline-color: inherit; clip: rect(1px, 1px, 1px, 1px); clip-path: inset(50%); height: 1px; width: 1px; overflow-wrap: normal; border: 0px; margin: -1px; padding: 0px; position: absolute; overflow: hidden;">Completed</span></span><span class="xp-tag-xp" style="box-sizing: inherit; outline-color: inherit; color: var(--theme-success-invert); margin-bottom: 1px; font-size: 12px;">100 XP</span>

- 3 minutes

Just as the INSERT statement always adds whole rows to a table, the DELETE statement always removes entire rows.

## Use DELETE to remove specific rows

DELETE operates on a set of rows, either defined by a condition in a WHERE clause or defined in a join. The WHERE clause in a DELETE statement has the same structure as a WHERE clause in a SELECT statement.

 Note

It’s important to keep in mind that a DELETE without a corresponding WHERE clause will remove all the rows from a table. Use the DELETE statement with caution.

The following code shows the basic syntax of the DELETE statement:

SQLCopy

```
DELETE [FROM] <TableName>
WHERE <search_conditions>;

```

The following example uses the DELETE statement to remove all products from the specified table that have been discontinued. There's a column in the table called _discontinued_ and for products that are no longer available, the column has a value of 1.

SQLCopy

```
DELETE FROM Production.Product
WHERE discontinued = 1;

```

## Use TRUNCATE TABLE to remove all rows

DELETE without a WHERE clause removes all the rows from a table. For this reason, DELETE is usually used conditionally, with a filter in the WHERE clause. If you really do want to remove all the rows and leave an empty table, you can use the TRUNCATE TABLE statement. This statement does not allow a WHERE clause and always removes all the rows in one operation. Here’s an example:

SQLCopy

```
TRUNCATE TABLE Sales.Sample;

```

TRUNCATE TABLE is more efficient than DELETE when you do want to remove all rows.

# Merge data based on multiple tables

<span class="xp-tag-icon is-shown-complete docon docon-check" style="box-sizing: inherit; outline-color: inherit; font-family: docons; font-size: 9px; speak: none; font-variant-numeric: normal; font-variant-east-asian: normal; text-align: center; direction: ltr; -webkit-font-smoothing: antialiased; line-height: 16px; display: inline-block; z-index: 3; color: var(--theme-success-invert); animation: 0.8s ease 0s 1 normal none running scaleIn; position: absolute; top: 1px; left: -5px;"><span class="visually-hidden" style="box-sizing: inherit; outline-color: inherit; clip: rect(1px, 1px, 1px, 1px); clip-path: inset(50%); height: 1px; width: 1px; overflow-wrap: normal; border: 0px; margin: -1px; padding: 0px; position: absolute; overflow: hidden;">Completed</span></span><span class="xp-tag-xp" style="box-sizing: inherit; outline-color: inherit; color: var(--theme-success-invert); margin-bottom: 1px; font-size: 12px;">100 XP</span>

- 3 minutes

In database operations, there is sometimes a need to perform a SQL MERGE operation. This DML option allows you to synchronize two tables by inserting, updating, or deleting rows in one table based on differences found in the other table. The table that is being modified is referred to as the _target_ table. The table that is used to determine which rows to change are called the _source_ table.

MERGE modifies data, based on one or more conditions:

- When the source data has a matching row in the target table, it can update data in the target table.
- When the source data has no match in the target, it can insert data into the target table.
- When the target data has no match in the source, it can delete the target data.

The general syntax of a MERGE statement is shown below. We're matching the target and the source on a specified column, and if there's a match between target and source, we specify an action to take on the target table. If there's not a match, we specify an action. The action can be an INSERT, UPDATE, or DELETE operation. This code indicates that an UPDATE is performed when there's a match between the source and the target. An INSERT is performed when there's data in the source with no matching data in the target. Finally, a DELETE is performed when there is data in the target with no match in the source. There are many other possible forms of a MERGE statement.

SQLCopy

```
MERGE INTO schema_name.table_name AS TargetTbl
USING (SELECT <select_list>) AS SourceTbl
ON (TargetTbl.col1 = SourceTbl.col1)
WHEN MATCHED THEN 
   UPDATE SET TargetTbl.col2 = SourceTbl.col2
WHEN NOT MATCHED [BY TARGET] THEN
   INSERT (<column_list>)
   VALUES (<value_list>)
WHEN NOT MATCHED BY SOURCE THEN
   DELETE;

```

You can use only the elements of the MERGE statement that you need. For example, suppose the database includes a table of staged invoice updates, that includes a mix of revisions to existing invoices and new invoices. You can use the WHEN MATCHED and WHEN NOT MATCHED clauses to update or insert invoice data as required.

SQLCopy

```
MERGE INTO Sales.Invoice as i
USING Sales.InvoiceStaging as s
ON i.SalesOrderID = s.SalesOrderID
WHEN MATCHED THEN
    UPDATE SET i.CustomerID = s.CustomerID,
                i.OrderDate = GETDATE(),
                i.PurchaseOrderNumber = s.PurchaseOrderNumber,
                i.TotalDue = s.TotalDue
WHEN NOT MATCHED THEN
    INSERT (SalesOrderID, CustomerID, OrderDate, PurchaseOrderNumber, TotalDue)
    VALUES (s.SalesOrderID, s.CustomerID, s.OrderDate, s.PurchaseOrderNumber, s.TotalDue);
```

> **Tip**: As you follow the instructions in this pane, whenever you see a <span class="typeText" title="Type Text" tabindex="0" role="button" style="color: rgb(0, 127, 0); cursor: pointer;">icon</span>, you can use it to copy text from the instruction pane into the virtual machine interface.

If you are prompted to sign in, log into the **Student** account with the password <span class="typeText" title="Type Text" tabindex="0" role="button" style="color: rgb(0, 127, 0); cursor: pointer;">Pa55w.rd</span>. If prompted to allow your PC to be discoverable, select **No**.

# Modify Data

In this lab, you'll insert, update, and delete data in the **adventureworks** database. For your reference, the following diagram shows the tables in the database (you may need to resize the pane to see them clearly).

![An entity relationship diagram of the adventureworks database](https://raw.githubusercontent.com/MicrosoftLearning/dp-080-Transact-SQL/master/Instructions/Labs/images/adventureworks-erd.png)

> **Note**: If you're familiar with the standard **AdventureWorks** sample database, you may notice that in this lab we are using a simplified version that makes it easier to focus on learning Transact-SQL syntax.

## Insert data

You use the **INSERT** statement to insert data into a table.

1. Start Azure Data Studio, and create a new query (you can do this from the **File** menu or on the _welcome_ page).
    
2. In the new **SQLQuery\_…** pane, use the **Connect** button to connect the query to the **AdventureWorks** saved connection.
    
3. In the query editor, enter the following code to create a new table named **SalesLT.CallLog**, which we'll use in this lab.
    
    ```
    CREATE TABLE SalesLT.CallLog
    (
        CallID int IDENTITY PRIMARY KEY NOT NULL,
        CallTime datetime NOT NULL DEFAULT GETDATE(),
        SalesPerson nvarchar(256) NOT NULL,
        CustomerID int NOT NULL REFERENCES SalesLT.Customer(CustomerID),
        PhoneNumber nvarchar(25) NOT NULL,
        Notes nvarchar(max) NULL
    );
    ```
    
4. Use the **⏵Run** button to run the code and create the table. Don't worry too much about the details of the **CREATE TABLE** statement - it creates a table with some fields that we'll use in subsequent tasks to insert, update, and delete data.
    
5. Create a new query, so you have two **SQLQuery\_…** panes, and in the new pane, enter the following code to query the **SalesLT.CallLog** you just created.
    
    ```
    SELECT * FROM SalesLT.CallLog;
    ```
    
6. Run the **SELECT** query and view the results, which show the columns in the new table but no rows, because the table is empty.
    
7. Switch back to the **SQLQuery\_…** pane containing the **CREATE TABLE** statement, and replace it with the following **INSERT** statement to insert a new row into the **SalesLT.CallLog** table.
    
    ```
    INSERT INTO SalesLT.CallLog
    VALUES
    ('2015-01-01T12:30:00', 'adventure-works\pamela0', 1, '245-555-0173', 'Returning call re: enquiry about delivery');
    ```
    
8. Run the query and review the message, which should indicate that 1 row was affected.
    
9. Switch to the **SQLQuery\_…** pane containing the **SELECT** query and run it. Note that the results contain the row you inserted. The **CallID** column is an _identity_ column that is automatically incremented (so the first row has the value **1**), and the remaining columns contain the values you specified in the **INSERT** statement
    
10. Switch back to the **SQLQuery\_…** pane containing the **INSERT** statement, and replace it with the following code to insert another row. This time, the **INSERT** statement takes advantage of the fact that the table has a default value defined for the **CallTime** field, and allows _NULL_ values in the **Notes** field.
    
    ```
    INSERT INTO SalesLT.CallLog
    VALUES
    (DEFAULT, 'adventure-works\david8', 2, '170-555-0127', NULL);
    ```
    
11. Run the query and review the message, which should indicate that 1 row was affected.
    
12. Switch to the **SQLQuery\_…** pane containing the **SELECT** query and run it. Note that the second row has been inserted, with the default value for the **CallTime** field (the current time when the row was inserted) and _NULL_ for the **Notes** field.
    
13. Switch back to the **SQLQuery\_…** pane containing the **INSERT** statement, and replace it with the following code to insert another row. This time, the **INSERT** statement explicitly lists the columns into which the new values will be inserted. The columns not specified in the statement support either default or _NULL_ values, so they can be omitted.
    
    ```
    INSERT INTO SalesLT.CallLog (SalesPerson, CustomerID, PhoneNumber)
    VALUES
    ('adventure-works\jillian0', 3, '279-555-0130');
    ```
    
14. Run the query and review the message, which should indicate that 1 row was affected.
    
15. Switch to the **SQLQuery\_…** pane containing the **SELECT** query and run it. Note that the third row has been inserted, once again using the default value for the **CallTime** field and _NULL_ for the **Notes** field.
    
16. Switch back to the **SQLQuery\_…** pane containing the **INSERT** statement, and replace it with the following code, which inserts two rows of data into the **SalesLT.CallLog** table.
    
    ```
    INSERT INTO SalesLT.CallLog
    VALUES
    (DATEADD(mi,-2, GETDATE()), 'adventure-works\jillian0', 4, '710-555-0173', NULL),
    (DEFAULT, 'adventure-works\shu0', 5, '828-555-0186', 'Called to arrange deliver of order 10987');
    ```
    
17. Run the query and review the message, which should indicate that 2 rows were affected.
    
18. Switch to the **SQLQuery\_…** pane containing the **SELECT** query and run it. Note that two new rows have been added to the table.
    
19. Switch back to the **SQLQuery\_…** pane containing the **INSERT** statement, and replace it with the following code, which inserts the results of a **SELECT** query into the **SalesLT.CallLog** table.
    
    ```
    INSERT INTO SalesLT.CallLog (SalesPerson, CustomerID, PhoneNumber, Notes)
    SELECT SalesPerson, CustomerID, Phone, 'Sales promotion call'
    FROM SalesLT.Customer
    WHERE CompanyName = 'Big-Time Bike Store';
    ```
    
20. Run the query and review the message, which should indicate that 2 rows were affected.
    
21. Switch to the **SQLQuery\_…** pane containing the **SELECT** query and run it. Note that two new rows have been added to the table. These are the rows that were retrieved by the **SELECT** query.
    
22. Switch back to the **SQLQuery\_…** pane containing the **INSERT** statement, and replace it with the following code, which inserts a row and then uses the **SCOPE\_IDENTITY** function to retrieve the most recent _identity_ value that has been assigned in the database (to any table), and also the **IDENT\_CURRENT** function, which retrieves the latest _identity_ value in the specified table.
    
    ```
    INSERT INTO SalesLT.CallLog (SalesPerson, CustomerID, PhoneNumber)
    VALUES
    ('adventure-works\josé1', 10, '150-555-0127');
    
    SELECT SCOPE_IDENTITY() AS LatestIdentityInDB,
           IDENT_CURRENT('SalesLT.CallLog') AS LatestCallID;
    ```
    
23. Run the code and review the results, which should be two numeric values, both the same.
    
24. Switch to the **SQLQuery\_…** pane containing the **SELECT** query and run it to validate that the new row that has been inserted has a **CallID** value that matches the _identity_ value returned when you inserted it.
    
25. Switch back to the **SQLQuery\_…** pane containing the **INSERT** statement, and replace it with the following code, which enables explicit insertion of _identity_ values and inserts a new row with a specified **CallID** value, before disabling explicit _identity_ insertion again.
    
    ```
    SET IDENTITY_INSERT SalesLT.CallLog ON;
    
    INSERT INTO SalesLT.CallLog (CallID, SalesPerson, CustomerID, PhoneNumber)
    VALUES
    (20, 'adventure-works\josé1', 11, '926-555-0159');
    
    SET IDENTITY_INSERT SalesLT.CallLog OFF;
    ```
    
26. Run the code and review the results, which should affect 1 row.
    
27. Switch to the **SQLQuery\_…** pane containing the **SELECT** query and run it to validate that a new row has been inserted with the especific **CallID** value you specified in the **INSERT** statement (9).
    

## Update data

To modify existing rows in a table, use the **UPDATE** statement.

1. On the **SQLQuery\_…** pane containing the **INSERT** statement, replace the existing code with the following code.
    
    ```
    UPDATE SalesLT.CallLog
    SET Notes = 'No notes'
    WHERE Notes IS NULL;
    ```
    
2. Run the **UPDATE** statement and review the message, which should indicate the number of rows affected.
    
3. Switch to the **SQLQuery\_…** pane containing the **SELECT** query and run it. Note that the rows that previously had _NULL_ values for the **Notes** field now contain the text _No notes_.
    
4. Switch back to the **SQLQuery\_…** pane containing the **UPDATE** statement, and replace it with the following code, which updates multiple columns.
    
    ```
    UPDATE SalesLT.CallLog
    SET SalesPerson = '', PhoneNumber = ''
    ```
    
5. Run the **UPDATE** statement and note the number of rows affected.
    
6. Switch to the **SQLQuery\_…** pane containing the **SELECT** query and run it. Note that _all_ rows have been updated to remove the **SalesPerson** and **PhoneNumber** fields - this emphasizes the danger of accidentally omitting a **WHERE** clause in an **UPDATE** statement.
    
7. Switch back to the **SQLQuery\_…** pane containing the **UPDATE** statement, and replace it with the following code, which updates the **SalesLT.CallLog** table based on the results of a **SELECT** query.
    
    ```
    UPDATE SalesLT.CallLog
    SET SalesPerson = c.SalesPerson, PhoneNumber = c.Phone
    FROM SalesLT.Customer AS c
    WHERE c.CustomerID = SalesLT.CallLog.CustomerID;
    ```
    
8. Run the **UPDATE** statement and note the number of rows affected.
    
9. Switch to the **SQLQuery\_…** pane containing the **SELECT** query and run it. Note that the table has been updated using the values returned by the **SELECT** statement.
    

## Delete data

To delete rows in the table, you generally use the **DELETE** statement; though you can also remove all rows from a table by using the **TRUNCATE TABLE** statement.

1. On the **SQLQuery\_…** pane containing the **UPDATE** statement, replace the existing code with the following code.
    
    ```
    DELETE FROM SalesLT.CallLog
    WHERE CallTime < DATEADD(dd, -7, GETDATE());
    ```
    
2. Run the **DELETE** statement and review the message, which should indicate the number of rows affected.
    
3. Switch to the **SQLQuery\_…** pane containing the **SELECT** query and run it. Note that rows with a **CallDate** older than 7 days have been deleted.
    
4. Switch back to the **SQLQuery\_…** pane containing the **DELETE** statement, and replace it with the following code, which uses the **TRUNCATE TABLE** statement to remove all rows in the table.
    
    ```
    TRUNCATE TABLE SalesLT.CallLog;
    ```
    
5. Run the **TRUNCATE TABLE** statement and note the number of rows affected.
    
6. Switch to the **SQLQuery\_…** pane containing the **SELECT** query and run it. Note that _all_ rows have been deleted from the table.
    

## Challenges

Now it's your turn to try modifying some data.

> **Tip**: Try to determine the appropriate code for yourself. If you get stuck, suggested answers are provided at the end of this lab.

### Challenge 1: Insert products

Each Adventure Works product is stored in the **SalesLT.Product** table, and each product has a unique **ProductID** identifier, which is implemented as an _identity_ column in the **SalesLT.Product** table. Products are organized into categories, which are defined in the **SalesLT.ProductCategory** table. The products and product category records are related by a common **ProductCategoryID** identifier, which is an _identity_ column in the **SalesLT.ProductCategory** table.

1. Insert a product
    - Adventure Works has started selling the following new product. Insert it into the **SalesLT.Product** table, using default or _NULL_ values for unspecified columns:
        - **Name**: LED Lights
        - **ProductNumber**: LT-L123
        - **StandardCost**: 2.56
        - **ListPrice**: 12.99
        - **ProductCategoryID**: 37
        - **SellStartDate**: _Today's date_
    - After you have inserted the product, run a query to determine the **ProductID** that was generated.
    - Then run a query to view the row for the product in the **SalesLT.Product** table.
2. Insert a new category with two products
    - Adventure Works is adding a product category for _Bells and Horns_ to its catalog. The parent category for the new category is **4** (_Accessories_). This new category includes the following two new products:
        - First product:
            - **Name**: Bicycle Bell
            - **ProductNumber**: BB-RING
            - **StandardCost**: 2.47
            - **ListPrice**: 4.99
            - **ProductCategoryID**: _The **ProductCategoryID** for the new Bells and Horns category_
            - **SellStartDate**: _Today's date_
        - Second product:
            - **Name**: Bicycle Horn
            - **ProductNumber**: BB-PARP
            - **StandardCost**: 1.29
            - **ListPrice**: 3.75
            - **ProductCategoryID**: _The **ProductCategoryID** for the new Bells and Horns category_
            - **SellStartDate**: _Today's date_
    - Write a query to insert the new product category, and then insert the two new products with the appropriate **ProductCategoryID** value.
    - After you have inserted the products, query the **SalesLT.Product** and **SalesLT.ProductCategory** tables to verify that the data has been inserted.

### Challenge 2: Update products

You have inserted data for a product, but the pricing details are not correct. You must now update the records you have previously inserted to reflect the correct pricing. Tip: Review the documentation for UPDATE in the Transact-SQL Language Reference.

1. Update product prices
    
    - The sales manager at Adventure Works has mandated a 10% price increase for all products in the _Bells and Horns_ category. Update the rows in the **SalesLT.Product** table for these products to increase their price by 10%.
2. Discontinue products
    
    - The new LED lights you inserted in the previous challenge are to replace all previous light products. Update the **SalesLT.Product** table to set the **DiscontinuedDate** to today’s date for all products in the Lights category (product category ID **37**) other than the LED Lights product you inserted previously.

### Challenge 3: Delete products

The Bells and Horns category has not been successful, and it must be deleted from the database.

1. Delete a product category and its products
    - Delete the records for the _Bells and Horns_ category and its products. You must ensure that you delete the records from the tables in the correct order to avoid a foreign-key constraint violation.

## Challenge Solutions

This section contains suggested solutions for the challenge queries.

### Challenge 1

1. Insert a product:
    
    ```
    INSERT INTO SalesLT.Product (Name, ProductNumber, StandardCost, ListPrice, ProductCategoryID, SellStartDate)
    VALUES
    ('LED Lights', 'LT-L123', 2.56, 12.99, 37, GETDATE());
    
    SELECT SCOPE_IDENTITY();
    
    SELECT * FROM SalesLT.Product
    WHERE ProductID = SCOPE_IDENTITY();
    ```
    
2. Insert a new category with two products:
    
    ```
    INSERT INTO SalesLT.ProductCategory (ParentProductCategoryID, Name)
    VALUES
    (4, 'Bells and Horns');
    
    INSERT INTO SalesLT.Product (Name, ProductNumber, StandardCost, ListPrice, ProductCategoryID, SellStartDate)
    VALUES
    ('Bicycle Bell', 'BB-RING', 2.47, 4.99, IDENT_CURRENT('SalesLT.ProductCategory'), GETDATE()),
    ('Bicycle Horn', 'BH-PARP', 1.29, 3.75, IDENT_CURRENT('SalesLT.ProductCategory'), GETDATE());
    
    SELECT c.Name As Category, p.Name AS Product
    FROM SalesLT.Product AS p
    JOIN SalesLT.ProductCategory as c
        ON p.ProductCategoryID = c.ProductCategoryID
    WHERE p.ProductCategoryID = IDENT_CURRENT('SalesLT.ProductCategory');
    ```
    

### Challenge 2

1. Update product prices:
    
    ```
    UPDATE SalesLT.Product
    SET ListPrice = ListPrice * 1.1
    WHERE ProductCategoryID =
        (SELECT ProductCategoryID
         FROM SalesLT.ProductCategory
         WHERE Name = 'Bells and Horns');
    ```
    
2. Discontinue products:
    
    ```
    UPDATE SalesLT.Product
    SET DiscontinuedDate = GETDATE()
    WHERE ProductCategoryID = 37
    AND ProductNumber <> 'LT-L123';
    ```
    

### Challenge 3

1. Delete a product category and its products:
    
    ```
    DELETE FROM SalesLT.Product
    WHERE ProductCategoryID =
        (SELECT ProductCategoryID
         FROM SalesLT.ProductCategory
         WHERE Name = 'Bells and Horns');
    
    DELETE FROM SalesLT.ProductCategory
    WHERE ProductCategoryID =
        (SELECT ProductCategoryID
         FROM SalesLT.ProductCategory
         WHERE Name = 'Bells and Horns');
    ```
    

## Return to Microsoft Learn

1. When you've finished the exercise, complete the [knowledge check](https://docs.microsoft.com/en-us/learn/modules/modify-data-with-transact-sql/8-knowledge-check) in Microsoft Learn.
2. When the link above opens in another browser tab, return to this one to end the lab environment.